In [1]:
pip install hausdorff

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for hausdorff: filename=hausdorff-0.2.6-py3-none-any.whl size=15194 sha256=7206a7f5b556b562a53d3c41c0335eec94d8b68dc526c9321417b1dc1c3fd105
  Stored in directory: c:\users\jsl3a\appdata\local\pip\cache\wheels\8d\56\ea\fd4bbacf1217e197e63de821a74730e953b23fdf2030733061
Successfully built hausdorff
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install numba

  Obtaining dependency information for numba from https://files.pythonhosted.org/packages/bc/43/a4a058fc1b58aa523b69141b98e51edf795fd0ff20e690dc8c1ed654c7fd/numba-0.57.1-cp39-cp39-win_amd64.whl.metadata
  Obtaining dependency information for llvmlite<0.41,>=0.40.0dev0 from https://files.pythonhosted.org/packages/e7/fb/a7430788e80cff1ec512ba6ca6d2159c0ed6293530b0d502d171e299872f/llvmlite-0.40.1-cp39-cp39-win_amd64.whl.metadata
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.5 MB 3.2 MB/s eta 0:00:01
   ----------- ---------------------------- 0.7/2.5 MB 5.8 MB/s eta 0:00:01
   -------------------- ------------------- 1.3/2.5 MB 7.3 MB/s eta 0:00:01
   ---------------------------- ----------- 1.8/2.5 MB 8.2 MB/s eta 0:00:01
   ------------------------------------ --- 2.3/2.5 MB 8.8 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 

In [14]:
import argparse
import os
import os.path as osp
import time

import numpy as np
import trimesh

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn

from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import torch_geometric.transforms as T
from torch_geometric.utils import from_trimesh, to_trimesh#, to_edge_index

# 1. Importing Mesh as Data

using the function from notebook: "2.ASMG Framework"

In [15]:
def trimesh2datalist(input_path):
    """read all trimesh from input_path, append to a list and return the list

    Args:
        input_path: The path to the trimesh files.

    Returns:
        a list with all the trimesh object in the path
    """
    #get item list to call functions on
    item_list = os.listdir(input_path)
    
    list = []

    #for all the item in input folder:
    for i in range(len(item_list)):
        in_path = input_path+"\\"+item_list[i]#set input path, start from folder
        mesh = trimesh.load(in_path)
        item = from_trimesh(mesh)
        list.append(item)
    
    return list

In [46]:
os.chdir("C:\\Users\jsl3a\Desktop\MSc Project Dataset\\nnUNet_raw\\Dataset001_amosCT")

start_time = time.time()

#load training and validation data into lists
ori_data = trimesh2datalist("version1/gb_stl_tr")
vgae_data = trimesh2datalist("version1/vgae_mesh")
recon_data = trimesh2datalist("version1/test")
#will do more in the future when we can implement baseline model & refinement loss
   
elapsed_time_min = (time.time() - start_time)/60
print(f"Elapsed time: {elapsed_time_min:.2f} minutes")

Elapsed time: 0.08 minutes


remove the template mesh from original data, make both list comparable:

In [47]:
ori_data = ori_data[:17] + ori_data[18:]
vgae_data = vgae_data[:17] + vgae_data[18:]

In [48]:
print(len(ori_data))
print(len(vgae_data))
print(len(recon_data))

162
162
162


We will randomly select 5 datapoints from the list for visualisation purpose:

In [49]:
# ori_list = []
# recon_list = []

# for i in range(5):
#     r = np.random.randint(0, len(ori_data))
#     ori_list.append(ori_data[r])
#     recon_list.append(recon_data[r])
#     print(r)

# 2. Evaluation

Although we were not able to recreate good synthetic shapes, we will still proceed to the evaluation stage, to complete this project.

The evaluation will again refer to Soodeh's paper, but will less metrics and aspects. We will focus on the **Generation Quality** and use the **Hausdorff Distance (HD)** & the **average of minimum Euclidean distance (ED)** for the metrics.

(p.s.) At the stage of writing we are not able to find code for the baseline model, the *rigid Registration-based Shape Matching (RSM)*, thus we don't have a base model yet.

## 2.1 Hausdorff Distance (HD)

The HD metric will make use of the `hausdorff_distance` function from the library `hausdorff` installed above. ([github](https://github.com/mavillan/py-hausdorff/tree/master))\
The function only takes numpy array as input, so we will have to convert the data from tensor to array before called the function over our data.

In [148]:
from statistics import mean, stdev
from hausdorff import hausdorff_distance

#function to convert list of tensor to list of numpy
def tensor2numpy(list):
    length = len(list)
    new_list = []
    for i in range(length):
        arr = list[i].pos.numpy()
        new_list.append(arr)
    return new_list

#function to call hausdorff_distance over whole dataset and return average distance/data
def hd_over_lists(list1, list2): #the list here is ndarray
    len1 = len(list1)
    len2 = len(list2)
    total_hd = 0
    
    if (len1 != len2):
        return None
    for i in range(len1):
        hd = hausdorff_distance(list1[i], list2[i])
        total_hd += hd
    return total_hd/len1

In [149]:
ori_datanp = tensor2numpy(ori_data)
vgae_datanp = tensor2numpy(vgae_data)
recon_datanp = tensor2numpy(recon_data)

In [155]:
hd0 = hd_over_lists(ori_datanp, vgae_datanp)
hd = hd_over_lists(ori_datanp, recon_datanp)
print(f"Hausdorff distance between original data and vgae recon data: {hd0:.4f}")
print(f"Hausdorff distance between original data and ASMG recon data: {hd:.4f}")

Hausdorff distance between original data and vgae recon data: 82.0366
Hausdorff distance between original data and ASMG recon data: 12890993379.5971


Again, the reconstructed data as seen in the report or previous notebook is not ideal (looks nothing like the gallbladder mesh), which explains the ginormous average value here after looping through the datasets. 

I brought in the vgae reconstructed mesh just for reference.
* it is not structurally normalised, just having the node features passed through the VGAE & recreated
* the same data was used for training, so it will overfit and have very low HD

## 2.2 Average Minimum Euclidean Distance (ED)

This metric is calclulated using the pytorch function `PairwiseDistance` ([doc](https://pytorch.org/docs/stable/generated/torch.nn.PairwiseDistance.html?highlight=distance)) where it defaults to computing the pairwise distance between input vectors, and the distance are computed using `p`-norm (defaulted to `p=2`).

In [151]:
#function to call ED calculation over whole dataset and return average distance/data
def avg_min_ed_over_lists(list1, list2, dim): #the list here is tensor
    len1 = len(list1)
    len2 = len(list2)
    total_ed = 0
    
    if (len1 != len2):
        return None
    for i in range(len1):
        in1 = list1[i].pos
        in2 = list2[i].pos
        ed = torch.cdist(in1, in2)
#         print(ed.shape)
#         print(i, in1.shape, in2.shape, torch.min(ed))
        min_by_0, _ = torch.min(ed, dim=dim)
#         print(min_by_0.shape)
        total_ed += torch.mean(min_by_0)
    return total_ed/len1

In [156]:
#0 is the dimension of the 2nd tensor

ed0 = avg_min_ed_over_lists(ori_data, vgae_data, 0)
ed = avg_min_ed_over_lists(ori_data, recon_data, 0)
print(f"ED between original data and vgae recon data: {ed0:.4f}")
print(f"ED between original data and ASMG recon data: {ed:.4f}")

ED between original data and vgae recon data: 57.6112
ED between original data and ASMG recon data: 448079040.0000


In [139]:
# #1 is the dimension of the 1st tensor
# ed0 = avg_min_ed_over_lists(ori_data, vgae_data, 1)
# ed = avg_min_ed_over_lists(ori_data, recon_data, 1)
# print(f"ED between original data and vgae recon data: {ed0:.4f}")
# print(f"ED between original data and ASMG recon data: {ed:.4f}")

The results here are interesting.

When we set the dimension in `torch.min` to `0`, which is the dimension ($|V_k^\prime|$) of reconstructed data, we get a more reasonable result where the ED from the ASMG reconstruction is way bigger than the ED from the VGAE reconstruction.

~~However, when we set the dimension to `1`, , which is the dimension ($|V_k|$) of original data,the ASMG reconstruction data actually have a lower ED than the VGAE reconstruction data.~~